In [ ]:
from common import *

with open_read('missing_luas.txt', True) as f:
    missing_modules = f.read().split('\n')
fake_missing_modules = '\n'.join([f'package.loaded["{module}"] = true' for module in missing_modules])
# print(fake_missing_modules)
module_not_found = re.compile(r"module '(.*?)' not found")

required_lua = []
config_path = 'resources/Scripts/Config/'
for cfg in ['Excel/CommonScriptConfig.lua', 'Json/ConfigEntity.lua']:
    with open_read(config_path + cfg) as f:
        required_lua.append(f.read())

def load_group(filename):
    with open(filename, 'r') as file:
        data = file.read()
    try:
        lua = lupa.LuaRuntime(unpack_returned_tuples=True)
        lua.execute(fake_missing_modules)
        for v in required_lua:
            lua.execute(v)
        # lua.execute(data.replace('require', 'print'))
        lua.execute(data)
        return lua.globals()
    except lupa.LuaError as e:
        s = str(e)
        for module in module_not_found.findall(s):
            return module
        return s.replace('"',"'")

In [ ]:
lua_gadgets = {}
lua_groups = {}
errors = {}
path = os.path.join(GC_PATH, 'resources/Scripts/Scene/')
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.lua'):
            full = os.path.join(root, file)
            key = full[len(path):]
            results = load_group(full)
            if isinstance(results, str):
                errors[key] = results
            else:
                gadgets = results.gadgets
                if gadgets is not None:
                    lua_gadgets[key] = table_to_dict(gadgets)
                groups = results.groups
                if groups is not None:
                    lua_groups[key] = table_to_dict(groups)
lua_gadgets['errors'] = errors
with open_write('gadgets.lua.json', True) as file:
    file.write(encode_json(lua_gadgets))
with open_write('groups.lua.json', True) as file:
    file.write(encode_json(lua_groups))
print(f'Total errors: {len(errors)}')

In [ ]:
for thing in ['gadgets', 'groups']:
    lua_gadgets_decoded = load_json(f'{thing}.lua.json', True)
    lua_gadgets_flat = {}
    for key, gadgets in lua_gadgets_decoded.items():
        if isinstance(gadgets, list):
            lua_gadgets_flat[key] = [flatten_json(g, flatten_xyz=True, flatten_lists=False) for g in gadgets]
        elif isinstance(gadgets, dict):
            lua_gadgets_flat[key] = flatten_json(gadgets, flatten_xyz=True, flatten_lists=False)
        else:
            lua_gadgets_flat[key] = gadgets
    with open_write(f'{thing}.flat.lua.json', True) as file:
        file.write(encode_json(lua_gadgets_flat, maxline=250, sort_overrides={'id': 'AAAid', 'pos': 'AApos'}))